# Introduction

In this notebook I will be exploring the text structure of Reddit whisky reviews stored at 

# Imports/Setup

In [1]:
import spacy

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
!sudo python -m spacy download en
nlp = spacy.load('en')

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /usr/local/anaconda/lib/python3.6/site-packages/en_core_web_sm -->
    /

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [3]:
import os
import pandas as pd
import numpy as np
import pymongo
import pyLDAvis
import pyLDAvis.gensim
from gensim.models import TfidfModel, LsiModel, LdaModel
from gensim.corpora import Dictionary
from lda_funcs import *
pyLDAvis.enable_notebook()

/usr/local/anaconda/lib/python3.6/site-packages/scipy/stats/morestats.py:12: DeprecationWarning: Importing from numpy.testing.decorators is deprecated, import from numpy.testing instead.
  from numpy.testing.decorators import setastest
/usr/local/anaconda/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Data Load

**Gensim Assets** 

In [4]:
gensim_dict = Dictionary.load(os.getenv('DOMINO_WORKING_DIR') + '/models/gensim_lemma.dictionary')

# WhiskyReviews Database Connection

In [5]:
client = pymongo.MongoClient("mongodb://whiskyadmin:%s@whiskyreviews-shard-00-00-mvfds.mongodb.net:27017,whiskyreviews-shard-00-01-mvfds.mongodb.net:27017,whiskyreviews-shard-00-02-mvfds.mongodb.net:27017/test?ssl=true&replicaSet=WhiskyReviews-shard-0&authSource=admin&retryWrites=true" % (os.getenv('mongopw')))
reddit_reviews = client.reddit_reviews
parsed = reddit_reviews['parsed']

In [6]:
documents = [x for x in parsed.find()]

In [7]:
len(documents)

31126

In [8]:
reviews = pd.DataFrame(documents)

In [9]:
corpus = [gensim_dict.doc2bow(text) for text in reviews.lemmas]

In [12]:
lda_initialscan = topic_model_scan(intext=reviews.lemmas, dictionary=gensim_dict, topic_counts=[1,2,5,7,9,10,12,15,20,50], split=False)

N Topics: 1	 Perplexity: -7.2367	 Coherence: 0.2566
N Topics: 2	 Perplexity: -7.1739	 Coherence: 0.2879
N Topics: 5	 Perplexity: -7.1113	 Coherence: 0.3219
N Topics: 7	 Perplexity: -7.0955	 Coherence: 0.3437
N Topics: 9	 Perplexity: -7.0789	 Coherence: 0.4012
N Topics: 10	 Perplexity: -7.0629	 Coherence: 0.3975
N Topics: 12	 Perplexity: -7.0514	 Coherence: 0.4086
N Topics: 15	 Perplexity: -7.046	 Coherence: 0.4088
N Topics: 20	 Perplexity: -7.0343	 Coherence: 0.4134
N Topics: 50	 Perplexity: -7.0278	 Coherence: 0.3899


In [13]:
review_lemmas_k5_ldavis, review_lemmas_k5_ldamod = lda_visgen(lda_initialscan, 5, refine=False)
review_lemmas_k7_ldavis, review_lemmas_k7_ldamod = lda_visgen(lda_initialscan, 7, refine=False)
review_lemmas_k9_ldavis, review_lemmas_k9_ldamod = lda_visgen(lda_initialscan, 9, refine=False)
review_lemmas_k15_ldavis, review_lemmas_k15_ldamod = lda_visgen(lda_initialscan, 15, refine=False)

In [14]:
review_lemmas_k5_ldavis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.007223  0.016449       1        1  25.363796
1     -0.136389 -0.061651       2        1  20.812111
2      0.003629  0.121571       3        1  21.391504
3      0.006109 -0.011684       4        1  17.473072
4      0.133873 -0.064684       5        1  14.959507, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
2280  Default  29217.000000        peat  29217.000000  30.0000  30.0000
55    Default  20275.000000         rye  20275.000000  29.0000  29.0000
790   Default  18110.000000        nbsp  18110.000000  28.0000  28.0000
659   Default  17846.000000           |  17846.000000  27.0000  27.0000
795   Default  28915.000000       smoke  28915.000000  26.0000  26.0000
73    Default  45405.000000      whisky  45405.000000  25.0000  25.0000
3073  Default  12509.000000           =  12509.000000  24.0000  24.0000
589   Default  28208.000000        malt  28208.000000  23.0000  23.0000
958   Default  36150.000000        cask  36150.000000  22.0000  22.0000
362   Default  46192.000000         oak  46192.000000  21.0000  21.0000
196   Default  11379.000000       proof  11379.000000  20.0000  20.0000
2523  Default   8970.000000   laphroaig   8970.000000  19.0000  19.0000
255   Default  26550.000000     bourbon  26550.000000  18.0000  18.0000
2645  Default   6972.000000    talisker   6972.000000  17.0000  17.0000
237   Default  12054.000000     whiskey  12054.000000  16.0000  16.0000
15    Default   8318.000000       blend   8318.000000  15.0000  15.0000
204   Default  47395.000000      review  47395.000000  14.0000  14.0000
27    Default   8655.000000     flavour   8655.000000  13.0000  13.0000
1218  Default   5419.000000      turkey   5419.000000  12.0000  12.0000
1035  Default  11573.000000  distillery  11573.000000  11.0000  11.0000
1409  Default  25827.000000      sherry  25827.000000  10.0000  10.0000
44    Default  31792.000000      palate  31792.000000   9.0000   9.0000
328   Default  35035.000000     vanilla  35035.000000   8.0000   8.0000
172   Default  15407.000000      medium  15407.000000   7.0000   7.0000
251   Default  13574.000000      barrel  13574.000000   6.0000   6.0000
447   Default  15343.000000   chocolate  15343.000000   5.0000   5.0000
1626  Default   6691.000000       grain   6691.000000   4.0000   4.0000
168   Default  48285.000000        like  48285.000000   3.0000   3.0000
56    Default  15544.000000      scotch  15544.000000   2.0000   2.0000
715   Default   6370.000000     however   6370.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
206    Topic5   5797.520996         say  13546.556641   1.0511  -5.0464
958    Topic5  11846.040039        cask  36150.640625   0.7841  -4.3319
21     Topic5   4609.950195      colour  10686.409180   1.0591  -5.2757
783    Topic5   3086.970215      butter   6401.466797   1.1705  -5.6767
204    Topic5  13453.512695      review  47395.812500   0.6405  -4.2046
57     Topic5   5011.523438         see  12945.051758   0.9508  -5.1921
43     Topic5  10101.592773         one  36493.316406   0.6154  -4.4912
231    Topic5   6972.730469        time  22317.417969   0.7365  -4.8619
168    Topic5  11878.329102        like  48285.664062   0.4974  -4.3292
73     Topic5  11163.431641      whisky  45405.132812   0.4968  -4.3912
170    Topic5   6067.110840        make  18430.783203   0.7887  -5.0010
31     Topic5   6561.122559          go  21769.212891   0.7005  -4.9227
396    Topic5   3303.599121   different   8059.618652   1.0080  -5.6089
65     Topic5  10118.012695       taste  54383.058594   0.2181  -4.4896
1011   Topic5   2996.121094         use   7249.316895   1.0162  -5.7066
296    Topic5   5570.088867        nice  21326.783203   0.5573  -5.0865
905    Topic5   4617.977539        dram  16209.0244

In [15]:
review_lemmas_k7_ldavis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.000107 -0.011252       1        1  21.719540
1     -0.167416  0.014855       2        1  16.584206
2      0.009929  0.111434       3        1  17.044832
3      0.043971  0.022280       4        1  13.188818
4      0.081057 -0.156998       5        1  11.083918
5      0.128214  0.067006       6        1  10.046743
6     -0.095861 -0.047326       7        1  10.331953, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
659   Default  25419.000000           |  25419.000000  30.0000  30.0000
2280  Default  30421.000000        peat  30421.000000  29.0000  29.0000
795   Default  29690.000000       smoke  29690.000000  28.0000  28.0000
55    Default  19726.000000         rye  19726.000000  27.0000  27.0000
73    Default  44928.000000      whisky  44928.000000  26.0000  26.0000
3073  Default  12338.000000           =  12338.000000  25.0000  25.0000
790   Default  17411.000000        nbsp  17411.000000  24.0000  24.0000
958   Default  36207.000000        cask  36207.000000  23.0000  23.0000
362   Default  45643.000000         oak  45643.000000  22.0000  22.0000
589   Default  28442.000000        malt  28442.000000  21.0000  21.0000
2523  Default   9304.000000   laphroaig   9304.000000  20.0000  20.0000
196   Default  11060.000000       proof  11060.000000  19.0000  19.0000
219   Default  29368.000000       spice  29368.000000  18.0000  18.0000
255   Default  25997.000000     bourbon  25997.000000  17.0000  17.0000
328   Default  35022.000000     vanilla  35022.000000  16.0000  16.0000
237   Default  11696.000000     whiskey  11696.000000  15.0000  15.0000
2272  Default   6689.000000       islay   6689.000000  14.0000  14.0000
15    Default   8048.000000       blend   8048.000000  13.0000  13.0000
204   Default  47197.000000      review  47197.000000  12.0000  12.0000
0     Default  19126.000000           $  19126.000000  11.0000  11.0000
447   Default  14759.000000   chocolate  14759.000000  10.0000  10.0000
44    Default  31241.000000      palate  31241.000000   9.0000   9.0000
2641  Default   6529.000000  springbank   6529.000000   8.0000   8.0000
114   Default  17093.000000       color  17093.000000   7.0000   7.0000
495   Default  16433.000000    cinnamon  16433.000000   6.0000   6.0000
1486  Default   6810.000000       brine   6810.000000   5.0000   5.0000
113   Default  11528.000000      cherry  11528.000000   4.0000   4.0000
1519  Default  11864.000000        salt  11864.000000   3.0000   3.0000
124   Default  14668.000000        dark  14668.000000   2.0000   2.0000
1035  Default  11597.000000  distillery  11597.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
1022   Topic7   1875.749878          30   3818.062500   1.5592  -5.8047
958    Topic7   9542.254883        cask  36207.816406   0.9364  -4.1780
44     Topic7   7154.841797      palate  31241.193359   0.7960  -4.4659
326    Topic7   3844.159912           u  12538.494141   1.0877  -5.0872
466    Topic7   5955.975586       score  24467.441406   0.8570  -4.6493
844    Topic7   1863.831299     network   4416.263672   1.4073  -5.8111
1      Topic7   8392.855469         100  45889.101562   0.5711  -4.3064
78     Topic7   4943.744629          10  20963.480469   0.8253  -4.8356
114    Topic7   4261.745605       color  17093.697266   0.8809  -4.9841
0      Topic7   4517.340332           $  19126.261719   0.8268  -4.9258
239    Topic7   4957.764160       would  24212.373047   0.6840  -4.8328
154    Topic7   6824.971680        good  44246.503906   0.4007  -4.5131
569    Topic7   1910.944092          20   4733.113281   1.3629  -5.7861
589    Topic7   5009.854492        malt  28442.343750   0.5335  -4.8223
1409   Topic7   4517.514648      sherry  25476.664062   0.5401  -4.9258
110    Topic7  

In [16]:
review_lemmas_k9_ldavis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.022836 -0.094322       1        1  16.808874
1     -0.092765  0.112243       2        1  12.613614
2      0.032619 -0.043277       3        1  13.276672
3     -0.026073 -0.115505       4        1  11.421518
4     -0.040111  0.004295       5        1  10.429395
5     -0.017140 -0.138390       6        1   8.913935
6     -0.075860  0.101922       7        1  12.755717
7      0.271450  0.085069       8        1   3.927188
8     -0.074956  0.087966       9        1   9.853083, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
659   Default  25114.000000           |  25114.000000  30.0000  30.0000
2280  Default  28900.000000        peat  28900.000000  29.0000  29.0000
795   Default  28202.000000       smoke  28202.000000  28.0000  28.0000
1409  Default  29842.000000      sherry  29842.000000  27.0000  27.0000
958   Default  40165.000000        cask  40165.000000  26.0000  26.0000
55    Default  19300.000000         rye  19300.000000  25.0000  25.0000
3073  Default  10682.000000           =  10682.000000  24.0000  24.0000
73    Default  44143.000000      whisky  44143.000000  23.0000  23.0000
154   Default  44587.000000        good  44587.000000  22.0000  22.0000
78    Default  20796.000000          10  20796.000000  21.0000  21.0000
790   Default  16722.000000        nbsp  16722.000000  20.0000  20.0000
0     Default  18808.000000           $  18808.000000  19.0000  19.0000
239   Default  24593.000000       would  24593.000000  18.0000  18.0000
362   Default  45047.000000         oak  45047.000000  17.0000  17.0000
2523  Default   8786.000000   laphroaig   8786.000000  16.0000  16.0000
589   Default  27926.000000        malt  27926.000000  15.0000  15.0000
328   Default  34369.000000     vanilla  34369.000000  14.0000  14.0000
255   Default  25698.000000     bourbon  25698.000000  13.0000  13.0000
219   Default  28819.000000       spice  28819.000000  12.0000  12.0000
196   Default  10787.000000       proof  10787.000000  11.0000  11.0000
572   Default  11071.000000         bad  11071.000000  10.0000  10.0000
824   Default   5122.000000     average   5122.000000   9.0000   9.0000
86    Default  13149.000000           5  13149.000000   8.0000   8.0000
447   Default  15351.000000   chocolate  15351.000000   7.0000   7.0000
2645  Default   6786.000000    talisker   6786.000000   6.0000   6.0000
2272  Default   6265.000000       islay   6265.000000   5.0000   5.0000
1519  Default  11397.000000        salt  11397.000000   4.0000   4.0000
124   Default  14884.000000        dark  14884.000000   3.0000   3.0000
821   Default  18934.000000       apple  18934.000000   2.0000   2.0000
364   Default  20023.000000      pepper  20023.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
2583   Topic9   2648.577148    macallan   4335.536133   1.8246  -5.4122
2677   Topic9   3253.091064      mature   5730.885742   1.7511  -5.2066
1110   Topic9    994.675293       prune   1311.138428   2.0412  -6.3916
883    Topic9   2391.582275  strawberry   4090.823242   1.7806  -5.5143
2303   Topic9   2886.864746   influence   5277.697266   1.7141  -5.3261
447    Topic9   6377.906738   chocolate  15351.599609   1.4390  -4.5334
2641   Topic9   3326.089844  springbank   6951.063965   1.5803  -5.1844
1291   Topic9   1251.026367         fig   1886.177734   1.9068  -6.1623
455    Topic9   7668.112305       fruit  28309.765625   1.0112  -4.3492
326    Topic9   4439.413574           u  12789.552734   1.2593  -4.8957
334    Topic9   8970.129883        year  37610.292969   0.8840  -4.1923
124    Topic9   4373.788574        dark  14884.431641   1.0927  -4.9106
1495   Topic9   2313.944092          ex   5450.296875   1.4607  -5.5473
665    Topic9   2088.477051       grape   4745.8203

In [17]:
review_lemmas_k15_ldavis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.077371 -0.146327       1        1   2.845853
1      0.086533  0.112898       2        1   3.966367
2     -0.123412  0.074227       3        1   2.890573
3      0.073801 -0.020900       4        1   5.650809
4      0.086110 -0.028293       5        1   6.438956
5     -0.241406 -0.061886       6        1   1.919195
6      0.069393  0.116266       7        1   5.429022
7     -0.236520  0.015430       8        1   1.633740
8     -0.008609  0.095220       9        1   6.940661
9      0.042533 -0.075415      10        1   9.259702
10    -0.032336 -0.018724      11        1  10.342083
11     0.041574  0.145751      12        1  13.660661
12     0.097670  0.008158      13        1   5.153319
13     0.071689 -0.118971      14        1   4.969449
14    -0.004391 -0.097434      15        1  18.899605, topic_info=     Category          Freq       Term         Total  loglift  logprob
term                                                                  
659   Default  18882.000000          |  18882.000000  30.0000  30.0000
2280  Default  28774.000000       peat  28774.000000  29.0000  29.0000
795   Default  28340.000000      smoke  28340.000000  28.0000  28.0000
3073  Default  11002.000000          =  11002.000000  27.0000  27.0000
55    Default  18763.000000        rye  18763.000000  26.0000  26.0000
790   Default  17474.000000       nbsp  17474.000000  25.0000  25.0000
0     Default  19159.000000          $  19159.000000  24.0000  24.0000
73    Default  43803.000000     whisky  43803.000000  23.0000  23.0000
154   Default  44246.000000       good  44246.000000  22.0000  22.0000
78    Default  20699.000000         10  20699.000000  21.0000  21.0000
239   Default  24589.000000      would  24589.000000  20.0000  20.0000
958   Default  36524.000000       cask  36524.000000  19.0000  19.0000
1409  Default  25749.000000     sherry  25749.000000  18.0000  18.0000
589   Default  28326.000000       malt  28326.000000  17.0000  17.0000
1     Default  45428.000000        100  45428.000000  16.0000  16.0000
204   Default  48426.000000     review  48426.000000  15.0000  15.0000
362   Default  44935.000000        oak  44935.000000  14.0000  14.0000
86    Default  13107.000000          5  13107.000000  13.0000  13.0000
326   Default  13876.000000          u  13876.000000  12.0000  12.0000
334   Default  37664.000000       year  37664.000000  11.0000  11.0000
881   Default   7238.000000       port   7238.000000  10.0000  10.0000
2523  Default   8689.000000  laphroaig   8689.000000   9.0000   9.0000
255   Default  25572.000000    bourbon  25572.000000   8.0000   8.0000
572   Default  10619.000000        bad  10619.000000   7.0000   7.0000
301   Default  26715.000000        old  26715.000000   6.0000   6.0000
2     Default  17328.000000         12  17328.000000   5.0000   5.0000
44    Default  31725.000000     palate  31725.000000   4.0000   4.0000
219   Default  28675.000000      spice  28675.000000   3.0000   3.0000
2645  Default   6900.000000   talisker   6900.000000   2.0000   2.0000
466   Default  24650.000000      score  24650.000000   1.0000   1.0000
...       ...           ...        ...           ...      ...      ...
239   Topic15  10539.214844      would  24589.998047   0.8188  -4.6825
236   Topic15   4019.324707       want   7668.007324   1.0201  -5.6465
16    Topic15   6303.614746        buy  13926.408203   0.8734  -5.1965
0     Topic15   8042.447754          $  19159.447266   0.7980  -4.9528
388   Topic15   4819.808105       burn  10229.927734   0.9134  -5.4648
31    Topic15   8513.245117         go  21766.160156   0.7273  -4.8960
61    Topic15   5379.068848  something  12102.478516   0.8551  -5.3551
431   Topic15   7405.662109        try  18896.628906   0.7293  -5.0353
168   Topic15  14774.453125       like  48018.683594   0.4873  -4.3447
67    Topic15   7252.645508      think  18593.857422   0.

# Overview

It looks like the Topic model for k=7 has the most effective separation of terms.  We will rerun that model with more epochs to see if it changes.

In [18]:
review_lemmas_k7_ldavis, review_lemmas_k7_ldamod = lda_visgen(lda_initialscan, 7, refine=True)

Model refinement requested, training new LDA model with 7 topics.


In [19]:
review_lemmas_k7_ldavis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.000107 -0.011252       1        1  21.717997
1     -0.167416  0.014855       2        1  16.584570
2      0.009929  0.111434       3        1  17.043812
3      0.043971  0.022280       4        1  13.188569
4      0.081057 -0.156998       5        1  11.084894
5      0.128214  0.067006       6        1  10.047593
6     -0.095861 -0.047326       7        1  10.332564, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
659   Default  25422.000000           |  25422.000000  30.0000  30.0000
2280  Default  30422.000000        peat  30422.000000  29.0000  29.0000
795   Default  29690.000000       smoke  29690.000000  28.0000  28.0000
55    Default  19724.000000         rye  19724.000000  27.0000  27.0000
73    Default  44928.000000      whisky  44928.000000  26.0000  26.0000
3073  Default  12338.000000           =  12338.000000  25.0000  25.0000
790   Default  17412.000000        nbsp  17412.000000  24.0000  24.0000
958   Default  36209.000000        cask  36209.000000  23.0000  23.0000
362   Default  45642.000000         oak  45642.000000  22.0000  22.0000
589   Default  28442.000000        malt  28442.000000  21.0000  21.0000
2523  Default   9304.000000   laphroaig   9304.000000  20.0000  20.0000
196   Default  11059.000000       proof  11059.000000  19.0000  19.0000
219   Default  29368.000000       spice  29368.000000  18.0000  18.0000
255   Default  25997.000000     bourbon  25997.000000  17.0000  17.0000
328   Default  35022.000000     vanilla  35022.000000  16.0000  16.0000
237   Default  11695.000000     whiskey  11695.000000  15.0000  15.0000
2272  Default   6689.000000       islay   6689.000000  14.0000  14.0000
15    Default   8048.000000       blend   8048.000000  13.0000  13.0000
204   Default  47197.000000      review  47197.000000  12.0000  12.0000
0     Default  19125.000000           $  19125.000000  11.0000  11.0000
447   Default  14759.000000   chocolate  14759.000000  10.0000  10.0000
44    Default  31240.000000      palate  31240.000000   9.0000   9.0000
2641  Default   6529.000000  springbank   6529.000000   8.0000   8.0000
114   Default  17093.000000       color  17093.000000   7.0000   7.0000
495   Default  16433.000000    cinnamon  16433.000000   6.0000   6.0000
1486  Default   6810.000000       brine   6810.000000   5.0000   5.0000
113   Default  11528.000000      cherry  11528.000000   4.0000   4.0000
1519  Default  11865.000000        salt  11865.000000   3.0000   3.0000
124   Default  14668.000000        dark  14668.000000   2.0000   2.0000
1035  Default  11597.000000  distillery  11597.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
1022   Topic7   1875.861938          30   3818.107422   1.5592  -5.8047
958    Topic7   9542.825195        cask  36209.449219   0.9363  -4.1780
44     Topic7   7155.269043      palate  31240.910156   0.7960  -4.4659
326    Topic7   3844.389404           u  12538.835938   1.0877  -5.0872
466    Topic7   5956.331543       score  24467.119141   0.8570  -4.6493
844    Topic7   1863.942749     network   4416.503418   1.4072  -5.8111
1      Topic7   8393.356445         100  45889.023438   0.5711  -4.3064
78     Topic7   4944.040039          10  20963.246094   0.8253  -4.8356
114    Topic7   4262.000488       color  17093.285156   0.8809  -4.9841
0      Topic7   4517.610352           $  19125.726562   0.8268  -4.9258
239    Topic7   4958.060059       would  24211.949219   0.6840  -4.8328
154    Topic7   6825.379395        good  44246.558594   0.4007  -4.5131
569    Topic7   1911.058228          20   4733.131836   1.3629  -5.7861
589    Topic7   5010.153809        malt  28442.382812   0.5335  -4.8223
1409   Topic7   4517.784668      sherry  25476.841797   0.5401  -4.9258
110    Topic7  

In [71]:
term_retriever(review_lemmas_k7_ldavis, lambda_val=0.6, top_n=15)

rank,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0
Category,,,,,,,,,,,,,,,
Default,|,peat,smoke,rye,whisky,=,nbsp,cask,oak,malt,laphroaig,proof,spice,bourbon,vanilla
Topic1,scotch,taste,smoke,like,100,12,review,bottle,10,sweet,peat,talisker,color,dram,get
Topic2,oak,spice,vanilla,pepper,dry,chocolate,note,sweet,dark,cinnamon,cherry,caramel,orange,spicy,medium
Topic3,rye,bourbon,proof,barrel,whiskey,turkey,bottle,good,wild,corn,review,like,oak,caramel,get
Topic4,whisky,malt,blend,grain,distillery,age,single,40,canadian,light,year,irish,note,make,would
Topic5,peat,smoke,laphroaig,islay,brine,cask,salt,lemon,kilchoman,colour,ash,whisky,old,ila,caol
Topic6,|,review,like,however,time,flavour,one,say,taste,nice,let,really,bad,thing,well
Topic7,=,nbsp,cask,springbank,100,palate,0,score,average,good,pay,10,u,$,would


So there are still a couple issues with nonbreaking spaces, pipes, and equal signs showing up in the document. Those are things we can clean up later. 

The most interesting result from this LDA modeling exercise is that it looks like we can probably filter down our modeling terms to those occuring with some probability in topics 2 and 5, especially if we are interested in modeling based on just tasting notes.  It would be good to extract the per-topic term probabilities from the model and apply them to reviews to see how different topics concentrate.  

### Apply Topic Model to Documents

Next we will apply the topic model to each of the reviews so we have some information about what topic each review belongs to.  I am hoping this will not be incredibly informative, unless some people are more direct in their reviews than others. (Really topic modeling was performed to get at the per-topic probabilities).

In [75]:
reviews_wtopics = apply_topics(model=review_lemmas_k7_ldamod, corpus=corpus, dataframe=reviews, col_prefix='k7lemmas')

In [77]:
reviews_wtopics.k7lemmas_dominant_topic.value_counts()

1    9774
3    6059
2    4897
7    2867
4    2770
5    2617
6    2142
Name: k7lemmas_dominant_topic, dtype: int64

In [78]:
reviews_wtopics.to_pickle(os.getenv('DOMINO_WORKING_DIR') + '/data/processed/reviews_wk7lemmatopics.pkl')

## Save off Gensim Models

In [57]:
review_lemmas_k7_ldamod.save(os.getenv('DOMINO_WORKING_DIR') + '/models/k7lemmas_refined.ldamod')

## Save off Per-Topic Probabilities

These don't exactly seem to correspond to per-topic probabilities for tokens since they don't sum to anywhere close to one.  Need to do more research about what they actually are. 

In [58]:
term_topic_matrix = review_lemmas_k7_ldamod.get_topics()

In [59]:
words_for_index = [gensim_dict.id2token[x] for x in range(0,8368)]

In [60]:
term_topic_matrix = pd.DataFrame(term_topic_matrix).transpose()

In [61]:
term_topic_matrix['token'] = words_for_index

In [62]:
term_topic_matrix.head()

,0,1,2,3,4,5,6,token
0,4.554040e-03,1.322508e-06,5.524675e-03,3.675863e-03,0.000018,6.960281e-05,7.256850e-03,$
1,1.059977e-02,1.049358e-02,5.802063e-03,1.320193e-03,0.006939,2.476366e-03,1.348264e-02,100
2,7.131748e-03,2.287706e-03,1.863797e-03,2.552788e-03,0.000242,1.064249e-03,1.178023e-03,12
3,1.385378e-07,1.545671e-07,9.113200e-08,1.811082e-07,0.000003,1.595077e-05,1.229628e-04,220
4,1.411301e-07,1.551792e-07,1.590261e-05,1.803219e-07,0.000028,2.635139e-07,2.672150e-07,35cl


In [63]:
term_topic_matrix[0].describe()

count    8.368000e+03
mean     1.194951e-04
std      5.533193e-04
min      1.381349e-07
25%      1.387261e-07
50%      7.791496e-06
75%      4.344652e-05
max      1.351153e-02
Name: 0, dtype: float64

In [64]:
term_topic_matrix['sum'] = term_topic_matrix.sum(axis=1)

In [65]:
term_topic_matrix['highest_prob_topic'] = term_topic_matrix[[0,1,2,3,4,5,6]].idxmax(axis=1) + 1

In [66]:
term_topic_matrix.head()

,0,1,2,3,4,5,6,token,sum,highest_prob_topic
0,4.554040e-03,1.322508e-06,5.524675e-03,3.675863e-03,0.000018,6.960281e-05,7.256850e-03,$,0.021100,7
1,1.059977e-02,1.049358e-02,5.802063e-03,1.320193e-03,0.006939,2.476366e-03,1.348264e-02,100,0.051114,7
2,7.131748e-03,2.287706e-03,1.863797e-03,2.552788e-03,0.000242,1.064249e-03,1.178023e-03,12,0.016321,1
3,1.385378e-07,1.545671e-07,9.113200e-08,1.811082e-07,0.000003,1.595077e-05,1.229628e-04,220,0.000143,7
4,1.411301e-07,1.551792e-07,1.590261e-05,1.803219e-07,0.000028,2.635139e-07,2.672150e-07,35cl,0.000045,5


In [67]:
term_topic_matrix.set_index('token', inplace=True)

In [68]:
term_topic_matrix.to_pickle(os.getenv('DOMINO_WORKING_DIR') + '/data/processed/k7lemmas_pertopicprobs.pkl')